# Challenge 4: Telemetry to the Rescue

## Find crash in telemetry data

You need to design a SQL query that returns the `car_number`, `driver_name` and the average `brake` & `speed` for the full second just before the crash. You should have the timestamp of the crash from the previous challenge.

> Keep in mind that the timestamp is in local time, whereas the telemetry data uses UTC.

In [ ]:
# Find the crash in telemetry data that is loaded into BQ
%%bigquery telemetry_during_crash 

DECLARE crash_timestamp TIMESTAMP DEFAULT '2024-05-11 13:42:06 UTC';
-- Remember that the CCTV footage is in local time (CEST, which is UTC+2) whereas the telemetry data is in UTC.

SELECT
    driver_name,
    ROUND(AVG(tv_speed),3) AS avg_speed_last_second,
    ROUND(AVG(tv_brake),3) AS avg_brake_last_second
FROM
    `concise-mesh-477218-a5.dataset_name.telemetry`
WHERE
    time_utc >= TIMESTAMP_SUB(crash_timestamp, INTERVAL 1 SECOND)
    AND time_utc <= crash_timestamp
GROUP BY
    driver_name

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
print(telemetry_during_crash.to_markdown())

|    | driver_name             |   avg_speed_last_second |   avg_brake_last_second |
|---:|:------------------------|------------------------:|------------------------:|
|  0 | ARON, Paul              |                  66.551 |                   0     |
|  1 | KING, Jordan            |                 165.46  |                   0     |
|  2 | BARNARD, Taylor         |                 176.806 |                   0     |
|  3 | ROWLAND, Oliver         |                 128.462 |                   1.05  |
|  4 | DA COSTA, António Félix |                 136.381 |                   0     |
|  5 | FENESTRAZ, Sacha        |                  92.994 |                   2     |
|  6 | GÜNTHER, Maximilian     |                  64.928 |                  98     |
|  7 | WEHRLEIN, Pascal        |                 145.025 |                   0     |
|  8 | NATO, Norman            |                  50.598 |                   2.444 |
|  9 | CASSIDY, Nick           |                  49.758 |       

## Find the drivers

Now we have the filtered & aggregated data, let's ask Gemini to tell us which two drivers were involved in the crash.

Design a prompt that determines **which drivers** were involved and **why the model thinks that**.

> Note that we're *appending the result of the query from the previous cell to your prompt*, so you can refer to that.

In [ ]:
import vertexai
from google import genai
from IPython.display import Markdown
vertexai.init()

In [ ]:
prompt = """
Analyze this telemetry to determine the core drivers involved.

For this incident, identify the two drivers whose telemetry shows that they were involved in the accident.
Explain why.

Data:
{data}
""" 

In [ ]:

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
response = client.models.generate_content(
    model="gemini-2.5-pro",
    contents = prompt.format(data=telemetry_during_crash.to_markdown())
)
Markdown(response.text)

Based on the telemetry data, the two drivers involved in the incident are:

1.  **Maximilian GÜNTHER**
2.  **Jake HUGHES**

### Explanation

The core indicators of an accident or emergency maneuver in this telemetry are the extremely high values for `avg_brake_last_second`.

1.  **Jake Hughes** has an `avg_brake_last_second` value of **100**. This represents maximum braking force, indicating the driver slammed on the brakes in an emergency evasive action.

2.  **Maximilian Günther** has an `avg_brake_last_second` value of **98**. This is also an extremely high, near-maximum value that signifies an emergency braking situation.

No other driver in the data shows this level of braking. Most drivers have a value of 0, and the few with non-zero values are comparatively negligible (e.g., Rowland at 1.05, Vandoorne at 13.6).

The combination of extremely high, near-identical braking values and similarly reduced speeds for Hughes (61.075) and Günther (64.928) strongly suggests they were in close proximity and both reacted to the same immediate event, which was the accident involving them.